In [1]:
# getCameraMatrix
# Imports
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
import pprint
from scipy.optimize import least_squares
os.chdir('/home/gokul/Projects/VisualOdometry/Oxford_dataset/')
from ReadCameraModel import ReadCameraModel
from UndistortImage import UndistortImage
from random import sample
from custom_functions import *
root = '/home/gokul/Projects/VisualOdometry/Oxford_dataset/model'
fx, fy, cx, cy, G_camera_image, LUT = ReadCameraModel(root)
K = np.array([[fx,0,cx],[0,fy,cy],[0,0,1]])

## load image list
root = '/home/gokul/Projects/VisualOdometry/Oxford_dataset/stereo/centre/'

impaths = []
for f in sorted(glob.glob(root+"*.png")):
    impaths.append(f)
    

ModuleNotFoundError: No module named 'custom_functions'

In [2]:
# getCameraMatrix
# Imports
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
import pandas as pd
import pprint
from scipy.optimize import least_squares
os.chdir('/home/gokul/Projects/VisualOdometry/Oxford_dataset/')
from ReadCameraModel import ReadCameraModel
from UndistortImage import UndistortImage
from random import sample
from custom_functions import *
root = '/home/gokul/Projects/VisualOdometry/Oxford_dataset/model'
fx, fy, cx, cy, G_camera_image, LUT = ReadCameraModel(root)
K = np.array([[fx,0,cx],[0,fy,cy],[0,0,1]])

## load image list
root = '/home/gokul/Projects/VisualOdometry/Oxford_dataset/stereo/centre/'

impaths = []
for f in sorted(glob.glob(root+"*.png")):
    impaths.append(f)
    
i = 100 
H0  = np.identity(4)
p_0 = np.array([[0, 0, 0, 1]]).T
flag = 0

data_points = []
for i in range(len(impaths)-1):
# for i in range(1):
    im1 = UndistortImage(cv2.cvtColor(cv2.imread(impaths[i],0), cv2.COLOR_BAYER_GR2RGB),LUT)
    im2 = UndistortImage(cv2.cvtColor(cv2.imread(impaths[i+1],0), cv2.COLOR_BAYER_GR2RGB),LUT)

    pts1,pts2 = findKeypoints(im1,im2)

    F,mask  = FmatrixRansac(pts1,pts2,False)
    E = getEssentialMatrix(K,F)
    
#         # only inliers
    pts1 = pts1[mask.ravel() == 1]
    pts2 = pts2[mask.ravel() == 1]
    retval, R, t, mask = cv2.recoverPose(E, pts1, pts2, K)
    

    #     pose = poseEstimation(K,E,pts1,pts2)
    pose = np.column_stack(R,t)    
    a = np.array([0, 0, 0, 1])
    H = np.vstack((pose, a))

    H0 = H0 @ H
    p_projection = H0 @ p_0

    print('x- ', p_projection[0])
    print('y- ', p_projection[2])
    
    data_points.append([p_projection[0][0], -p_projection[2][0]])
    plt.scatter(p_projection[0][0], -p_projection[2][0], color='r')

    if cv2.waitKey(0) == 27:
        break
    flag = flag + 1

    cv2.destroyAllWindows()

plt.show()
pd.to_csv('Predefined.csv')

error: OpenCV(3.4.8) /io/opencv_contrib/modules/xfeatures2d/src/sift.cpp:1207: error: (-213:The function/feature is not implemented) This algorithm is patented and is excluded in this configuration; Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in function 'create'
